In [1]:
import pandas as pd
import plotly.express as px 
import plotly.graph_objects as go

import dash  
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_treeview_antd
import dash_bootstrap_components as dbc

In [2]:
#!pip install dash_treeview_antd

In [3]:
pd.set_option('display.max_columns',None)

In [4]:
# Import and clean data (importing csv into pandas)
#df = pd.read_csv("MM_completed_phase2n3.csv")
df = pd.read_csv("MM_completed_ph2n3_synth.csv")

df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,nct_number,title,acronym,status,study_results,conditions,interventions,outcome_measures,lineoftherapy,region,minage,maxage,sponsor/collaborators,gender,age,phases,enrollment,funded_bys,study_type,study_designs,other_ids,start_date,primary_completion_date,completion_date,first_posted,results_first_posted,last_update_posted,locations,study_documents,url
0,NCT00657488,Thalidomide 100 mg/Day Versus Thalidomide 400 ...,NaN,Completed,No Results Available,Multiple Myeloma,Drug: Thalidomide,all cause mortality|response rate,1L/2L+,US-only,18,65,Celgene,All,"18 Years and older (Adult, Older Adult)",Phase 2|Phase 3,400.0,Industry,Interventional,Allocation: Randomized|Intervention Model: Par...,LATH0102/IFM0102,"December 1, 2001","October 1, 2005","April 1, 2006","April 14, 2008",NaN,"October 18, 2019","Hopital St Joseph, Gilly, Belgium|UCL de Mont ...",NaN,https://ClinicalTrials.gov/show/NCT00657488
1,NCT02419118,"""Monoclonal Antibodies for Treatment of Multip...",DARA,Completed,No Results Available,Multiple Myeloma,Drug: Daratumumab|Drug: Lenalidomide|Drug: Dex...,Response by IMWG criteria|T cell subset numbers,1L/2L+,North-America,18,65,Vejle Hospital,All,"18 Years and older (Adult, Older Adult)",Phase 2|Phase 3,4.0,Other,Interventional,Allocation: Randomized|Intervention Model: Par...,181114,January 2015,"October 1, 2015","October 1, 2015","April 17, 2015",NaN,"November 7, 2017","Department of Hematology Vejle Hospital, Vejle...",NaN,https://ClinicalTrials.gov/show/NCT02419118
2,NCT00038090,Thalidomide-Dexamethasone for Multiple Myeloma,NaN,Completed,No Results Available,Multiple Myeloma,Drug: Thalidomide|Drug: Dexamethasone,Response Rate,1L/2L+,Latin-America,18,75,M.D. Anderson Cancer Center|Celgene Corporation,All,"Child, Adult, Older Adult",Phase 2|Phase 3,83.0,Other|Industry,Interventional,Allocation: Non-Randomized|Intervention Model:...,ID00-070,June 2000,May 2005,May 2005,"May 29, 2002",NaN,"October 31, 2018",University of Texas M. D. Anderson Cancer Cent...,NaN,https://ClinicalTrials.gov/show/NCT00038090
3,NCT00514371,A Study of Tanespimycin (KOS-953) in Patients ...,TIME-2,Completed,No Results Available,Multiple Myeloma,Drug: tanespimycin|Drug: Bortezomib,Objective Response Rate|Response rate between ...,1L/2L+,Europe,18,82,Bristol-Myers Squibb,All,"18 Years and older (Adult, Older Adult)",Phase 2|Phase 3,22.0,Industry,Interventional,Allocation: Randomized|Intervention Model: Par...,CA200-003|KAG-302,August 2007,February 2009,February 2009,"August 9, 2007",NaN,"December 7, 2015","Local Institution, Berkeley, California, Unite...",NaN,https://ClinicalTrials.gov/show/NCT00514371
4,NCT00090493,Study of MAGE-A3 and NY-ESO-1 Immunotherapy in...,NaN,Completed,Has Results,Multiple Myeloma,Biological: MAGE-A3|Biological: MAGE-A3 AND NY...,The Number of Participants Experiencing a Resp...,1L/2L+,Middle-East,8,60,University of Arkansas,All,"18 Years to 70 Years (Adult, Older Adult)",Phase 2|Phase 3,4.0,Other,Interventional,Allocation: N/A|Intervention Model: Single Gro...,UARK 2003-26,June 2004,July 2007,May 2012,"August 31, 2004","June 24, 2013","June 24, 2013",University of Arkansas for Medical Sciences/MI...,NaN,https://ClinicalTrials.gov/show/NCT00090493


In [5]:
df['completion_year'] = pd.DatetimeIndex(df['completion_date']).year
df.completion_year = df.completion_year.apply(lambda x: str(x).replace('.0',''))

In [6]:
df['industry_sponsor'] = df.funded_bys.str.contains('Industry').astype(int)
df.industry_sponsor.head()

0    1
1    0
2    1
3    1
4    0
Name: industry_sponsor, dtype: int64

In [7]:
import datetime

In [8]:
#app = dash.Dash(__name__)

In [34]:
app = dash.Dash(external_stylesheets = [dbc.themes.BOOTSTRAP])

In [36]:
dropdown = dbc.FormGroup(
    [
        dbc.Label("Dropdown", html_for="dropdown"),
        dcc.Dropdown(
            id="dropdown",
            options=[
                {"label": "Option 1", "value": 1},
                {"label": "Option 2", "value": 2},
            ],
        ),
    ]
)

phase = dbc.FormGroup([
                dbc.Label("Phase", html_for="dropdown"),
                dcc.Dropdown(id="select_phase",
                 options=[
                     {"label": "Phase 1", "value": "Phase 1"},
                     {"label": "Phase 1/2", "value": "Phase 1|Phase 2"},
                     {"label": "Phase 2", "value": "Phase 2"},
                     {"label": "Phase 2/3", "value": "Phase 2|Phase 3"},
                     {"label": "Phase 3", "value": "Phase 3"},
                     ],),
])

lot = dbc.FormGroup([
                dbc.Label("Line of Therapy", html_for="dropdown"),
                dcc.Dropdown(id="select_lot",
                 options=[
                     {"label": "Any Line of Therapy", "value": "All"},
                     {"label": "First-line", "value": "1L"},
                     {"label": "Second-line or higher", "value": "2L"},
                     ],),
])

spon = dbc.FormGroup([
                dbc.Label("Sponsor Type", html_for="dropdown"),
                dcc.Dropdown(id="select_spons",
                 options=[
                     {"label": "All Sponsor Types", "value": "All"},
                     {"label": "Industry Sponsors", "value": "Industry"}
                     ],),
])


age_slider = dbc.FormGroup(
    [
        dbc.Label("Age Group", html_for="range-slider"),
        dcc.RangeSlider(
                min=0,
                max=100,
                step=1,
                marks={
                    0: '0 ',
                    18: '18',
                    40: '40',
                    65: '65',
                    100: '100'
                },
                value=[18, 100],
            ),
    ]
)

region = dbc.FormGroup(
    [
        dbc.Label("Region", html_for="range-slider"),
        dbc.FormGroup([
                        dash_treeview_antd.TreeView(
                            multiple=False,
                            checkable=True,
                            checked=[],
                            selected=[],
                            expanded=['0'],
                            data={
                                'title': 'Europe','key': '0',
                                'children': [
                                    {"title": "Western-Europe", "key": "0-0-1"},
                                    {"title": "Eastern-Europe", "key": "0-0-2"},
                                    {"title": "Nordic-Countries", "key": "0-0-3"},
                                ],
                            }),
                        dash_treeview_antd.TreeView(
                            multiple=False,
                            checkable=True,
                            checked=[],
                            selected=[],
                            expanded=['0'],
                            data={
                                'title': 'Asia','key': '1',
                                'children': [
                                    {"title": "Western-Asia", "key": "1-0-1"},
                                    {"title": "Eastern-Asia", "key": "1-0-2"},
                                    {"title": "Central-Asia", "key": "1-0-3"},
                                 ],
                            }),
                        dash_treeview_antd.TreeView(
                            multiple=False,
                            checkable=True,
                            checked=[],
                            selected=[],
                            expanded=['0'],
                            data={
                                'title': 'Americas','key': '2',
                                'children': [
                                    {"title": "United States", "key": "2-0-1"},
                                    {"title": "Northen-America", "key": "2-0-2"},
                                    {"title": "Central-America", "key": "2-0-3"},
                                    {"title": "South-America", "key": "2-0-4"},
                                 ],
                            })
                    ])
    ]
)

row1 = html.Div(
    [               
        html.Br(),
        dbc.Row([
            dbc.Col([
                #html.Label('Study Phase'),
                dcc.Dropdown(id="slct_phase",
                 options=[
                     {"label": "Phase 1", "value": "Phase 1"},
                     {"label": "Phase 1/2", "value": "Phase 1|Phase 2"},
                     {"label": "Phase 2", "value": "Phase 2"},
                     {"label": "Phase 2/3", "value": "Phase 2|Phase 3"},
                     {"label": "Phase 3", "value": "Phase 3"}
                     ],
                 multi=False,
                 placeholder="Select Phase",
                 style={'width': "200px"}
                 ),
            ], width={"order": "first"}),

            dbc.Col([
                dcc.Dropdown(id="slct_spons",
                 options=[
                     {"label": "All Sponsor Types", "value": "All"},
                     {"label": "Industry Sponsors", "value": "Industry"}],
                 multi=False,
                 value="All",
                 placeholder="Select type of trial sponsor",
                 style={'width': "200px"}
                 ),
            ]),
            dbc.Col([
                dcc.Dropdown(id="slct_lot",
                 options=[
                     {"label": "Any Line of Therapy", "value": "All"},
                     {"label": "First-line", "value": "1L"},
                     {"label": "Second-line or higher", "value": "2L"}],
                 multi=False,
                 value="All",
                 placeholder="Select the line of therapy",
                 style={'width': "200px"}
                 ),
            ]),
            dbc.Col([
                dbc.DropdownMenu([
                dbc.Form([
                    html.Label('Select Region'),
                    dbc.FormGroup([
                        dash_treeview_antd.TreeView(
                            multiple=False,
                            checkable=True,
                            checked=[],
                            selected=[],
                            expanded=['0'],
                            data={
                                'title': 'Europe','key': '0',
                                'children': [
                                    {"title": "Western-Europe", "key": "0-0-1"},
                                    {"title": "Eastern-Europe", "key": "0-0-2"},
                                    {"title": "Nordic-Countries", "key": "0-0-3"},
                                ],
                            }),
                        dash_treeview_antd.TreeView(
                            multiple=False,
                            checkable=True,
                            checked=[],
                            selected=[],
                            expanded=['0'],
                            data={
                                'title': 'Asia','key': '1',
                                'children': [
                                    {"title": "Western-Asia", "key": "1-0-1"},
                                    {"title": "Eastern-Asia", "key": "1-0-2"},
                                    {"title": "Central-Asia", "key": "1-0-3"},
                                 ],
                            }),
                        dash_treeview_antd.TreeView(
                            multiple=False,
                            checkable=True,
                            checked=[],
                            selected=[],
                            expanded=['0'],
                            data={
                                'title': 'Americas','key': '2',
                                'children': [
                                    {"title": "United States", "key": "2-0-1"},
                                    {"title": "Northen-America", "key": "2-0-2"},
                                    {"title": "Central-America", "key": "2-0-3"},
                                    {"title": "South-America", "key": "2-0-4"},
                                 ],
                            })
                    ])#, row=True, className="pl-3 pr-4"
            ])
            ], style={'width': "200px"})
            ]),
            

            
        ],style={'width': '100%'}),
    ],style={'width': '90%'})

row2 = html.Div([
     dbc.Row([
            dbc.Col([
            
            ]), 
            dbc.Card(dbc.CardBody([
                        html.H5("Studies", className="card-title"),
                        html.H4("32,500"),
            ]), style={"width": "190px"},),
            
            
            dbc.Col([
            
            ]), 
            
            dbc.Card(dbc.CardBody([
                        html.H5("Subjects", className="card-title"),
                        html.H3("9,500,000"),
            ]), style={"width": "190px"},),
                
            dbc.Col([
            
            ]),
            
            dbc.Card(dbc.CardBody([
                        html.H5("Sites", className="card-title"),
                        html.H4("750,000"),
            ]), style={"width": "190px"},),
        ])
     ])

row3 = html.Div([
   # dbc.Row([
         html.Label('Age Group (use slider to select min and max age)'),
            dcc.RangeSlider(
                min=0,
                max=100,
                step=1,
                marks={
                    0: '0 ',
                    18: '18',
                    40: '40',
                    65: '65',
                    100: '100'
                },
                value=[18, 100],
            ),
     #    ])
    ])

app.layout = dbc.Container([html.H1("Clinical Trial Browser", style={'text-align': 'center'}),
                            html.Div([
                                html.Br(),
                                phase,
                                #html.Br(),
                                lot,
                                #html.Br(),
                                spon,
                                #html.Br(),
                                age_slider,
                                region,
                                html.Br()
                                ], style={'width': '28%', 'display': 'inline-block'}),
                            html.Div([
                                ], style={'width': '5%', 'display': 'inline-block'}),
                            html.Div([
                               #row1,
                               html.Br(),
                               row2,
                               html.Br(),
                               #row3
                            ], style={'width': '67%', 'display': 'inline-block', 'vertical-align': 'top'}),
                           ])

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2020 12:07:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2020 12:07:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2020 12:07:12] "GET /_dash-layout HTTP/1.1" 200 -


In [41]:
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    html.Div([
        
        ###### works below
        html.Label('Study Phase'),
        dcc.Dropdown(id="slct_phase",
                 options=[
                     {"label": "Phase 2", "value": "Phase 2"},
                     {"label": "Phase 3", "value": "Phase 3"},
                     {"label": "Phase 1/2", "value": "Phase 1|Phase 2"},
                     {"label": "Phase 2/3", "value": "Phase 2|Phase 3"}],
                 multi=False,
                 #value="Phase 2",
                 placeholder="Select phase of the trials",
                 style={'width': "50%"}
                 ),
        
        html.Label('Sponsor Type'),
        dcc.Dropdown(id="slct_spons",
                 options=[
                     {"label": "All Sponsor Types", "value": "All"},
                     {"label": "Industry Sponsors", "value": "Industry"}],
                 multi=False,
                 #value="All",
                 placeholder="Select type of trial sponsor",
                 style={'width': "50%"}
                 ),
        
        html.Label('Line of Therapy'),
        dcc.Dropdown(id="slct_lot",
                 options=[
                     {"label": "Any Line of Therapy", "value": "All"},
                     {"label": "First-line", "value": "1L"},
                     {"label": "Second-line or higher", "value": "2L"}],
                 multi=False,
                 value="All",
                 placeholder="Select the line of therapy",
                 style={'width': "50%"}
                 ),
        
        html.Label('Region'),
        dcc.Dropdown(id="slct_rgn",
                 options=[
                     {"label": "Any", "value": "All"},
                     [{"label": "North-America", "value": "North-America"},
                     {"label": "Central-America", "value": "Central-America"}],
                     {"label": "Europe", "value": "Europe"},
                     {"label": "East-Asia", "value": "East-Asia"},
                     {"label": "South-Asia", "value": "South-Asia"},
                     {"label": "Middle-East", "value": "Middle-East"},
                     {"label": "United States", "value": "US-only"}],
                 multi=True,
                 value="All",
                 placeholder="Select the geographical region",
                 style={'width': "40%"}
                 ),
        
        dbc.DropdownMenu([
            dbc.Form([
                dbc.FormGroup([
                    dash_treeview_antd.TreeView(
                        multiple=False,
                        checkable=True,
                        checked=[],
                        selected=[],
                        expanded=['0'],
                        data={
                            'title': 'Europe','key': '0',
                            'children': [
                                {"title": "Western-Europe", "key": "0-0-1"},
                                {"title": "Eastern-Europe", "key": "0-0-2"},
                                {"title": "Nordic-Countries", "key": "0-0-3"},
                            ],
                        }),
                    dash_treeview_antd.TreeView(
                        multiple=False,
                        checkable=True,
                        checked=[],
                        selected=[],
                        expanded=['0'],
                        data={
                            'title': 'Asia','key': '1',
                            'children': [
                                {"title": "Western-Asia", "key": "1-0-1"},
                                {"title": "Eastern-Asia", "key": "1-0-2"},
                                {"title": "Central-Asia", "key": "1-0-3"},
                             ],
                        }),
                    dash_treeview_antd.TreeView(
                        multiple=False,
                        checkable=True,
                        checked=[],
                        selected=[],
                        expanded=['0'],
                        data={
                            'title': 'Americas','key': '2',
                            'children': [
                                {"title": "United States", "key": "2-0-1"},
                                {"title": "Northen-America", "key": "2-0-2"},
                                {"title": "Central-America", "key": "2-0-3"},
                                {"title": "South-America", "key": "2-0-4"},
                             ],
                        })
                ], row=True, className="pl-3 pr-4")
            ])
        ]),
        
        html.Label('Age Group'),
        dcc.RangeSlider(
            min=0,
            max=100,
            step=1,
            marks={
                0: '0 ',
                18: '18',
                40: '40',
                65: '65',
                100: '100'
            },
            value=[18, 100],
        ),
        
    ]),
    #], style={'columnCount': 2}),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='trial_en_graph', figure={})

])


# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='trial_en_graph', component_property='figure')],
     #Output(component_id='output_container_slide', component_property='children')],
    [Input(component_id='slct_phase', component_property='value'),
     Input(component_id='slct_spons', component_property='value')]
     #Input(component_id='range_slider', component_property='value')]
)

# Business logic
def update_graph(option_slctd, spons_slctd):
    print(option_slctd)
    print(spons_slctd)

    container = "The phase chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["phases"] == option_slctd]
    #dff = dff[dff["phases"].isin(list(option_slctd))]
    if spons_slctd=='Industry':
        dff = dff[dff["industry_sponsor"] == 1]
    dff = dff[['completion_year','enrollment']].groupby('completion_year',as_index=False).sum()
    print(dff.head())
    
    fig = px.bar(dff, y='enrollment', x='completion_year')
    
    #container_slider = 'You have selected "{}"'.format(range_slider)
    
    return container, fig


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Sep/2020 19:33:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2020 19:33:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2020 19:33:24] "GET /_dash-layout HTTP/1.1" 200 -


None
None
Empty DataFrame
Columns: [completion_year, enrollment]
Index: []


127.0.0.1 - - [15/Sep/2020 19:33:24] "POST /_dash-update-component HTTP/1.1" 200 -
